If you use this script for your research, please cite our paper: Ciancetta A, Gill A. K., Ding T., Karlov D.S., McCormick P., Tikhonova I.G. 'Probe Confined Dynamic Mapping for GPCRs Allosteric Site Prediction' eLife, 2021, Submitted.

Initialize HTMD

In [1]:
from htmd.ui import *
from htmd.molecule.util import uniformRandomRotation
import nglview as nv
from nglview.datafiles import PDB, XTC
import configparser
from itertools import groupby
import math


ffevaluate module is in beta version

Please cite HTMD: Doerr et al.(2016)JCTC,12,1845. https://dx.doi.org/10.1021/acs.jctc.6b00049

HTMD Documentation at: https://www.htmd.org/docs/latest/



2021-05-27 16:23:57,411 - binstar - INFO - Using Anaconda API: https://api.anaconda.org


New devel HTMD version (1.24.7 python[3.6,<3.7.0a0,3.7,<3.8.0a0]) is available. You are currently on (1.13.9). Use 'conda update -c acellera htmd' to update to the new version. You might need to update your python version as well if there is no release for your current version.



/usr/local/miniconda3/lib/python3.6/site-packages/pyemma/__init__.py:91: UserWarning: You are not using the latest release of PyEMMA. Latest is 2.5.7, you have 2.4.
  .format(latest=latest, current=current), category=UserWarning)


## Load variables from Input File

In [2]:
cfg = configparser.RawConfigParser()
cfg.read('midas_acemd.cfg')       # Read file
par=dict(cfg.items("Settings"))
for p in par:
    par[p]=par[p].split("#",1)[0].strip() # To get rid of inline comments
globals().update(par)  #Make them availible globally
print (par)

{'name': 'hP2Y1_4XNW_superposed', 'lipid': 'POPC', 'coname': 'P2O', 'pm': '95.101', 'conc': '10', 'csolv': 'P2O', 'zoff': '6.000000', 'zheight': '20.0000', 'offset': '1.000000', 'zoffset': '1.000000', 'disu': '42-296; 124-202'}


## Chek if Packmol Runs

In [3]:
pmol = shutil.which("packmol", mode=os.X_OK)
if not pmol:
    raise FileNotFoundError('packmol not found. You should have packmol installed ')

## Calculate Cosolvent Box Dimensions

In [4]:
mol1 = Molecule(name + '.pdb')
c=mol1.get('coords', sel='protein and z > 0')
#Calculate minimum and maximum coordinates
pmin=np.min(c, axis=0)
pmax=np.max(c, axis=0)
xmin=pmin[0] 
xmax=pmax[0] 
ymin=pmin[1]
ymax=pmax[1] 
zmin=pmin[2]
zmax=pmax[2]
xsize=xmax-xmin 
ysize=ymax-ymin
zl2=zmax + float(zoff)
zl4=zl2+ float(zheight)
print(xsize,ysize,zl2,zl4)

38.921997 40.458 47.07899856567383 67.07899856567383


/usr/local/miniconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:200: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(mol._dtypes[field], np.float) \


## Visualize box

In [5]:
mol1.view('not water', gui=False)
b = VMDBox([xmin, xmax, ymin, ymax, zl2, zl4])

## Calculate number of water molecules fitting the box

In [6]:
#Create a box of soley water molecules
smol1 = solvate(mol1, minmax=[[xmin, ymin, zl2],[xmax, ymax, zl4]])
smol1.view()
#Calculate the number of water molecules fitting the box
numwat=len(smol1.get("resid",sel="name OH2"))
print(numwat)

2021-05-27 16:26:29,941 - htmd.builder.solvate - INFO - Using water pdb file at: /usr/local/miniconda3/lib/python3.6/site-packages/htmd/builder/wat.pdb
2021-05-27 16:26:31,436 - htmd.builder.solvate - INFO - Replicating 1 water segments, 1 by 1 by 1
Solvating: 100%|██████████| 1/1 [00:00<00:00,  1.19it/s]
2021-05-27 16:26:32,346 - htmd.builder.solvate - INFO - 961 water molecules were added to the system.


961


## Calculate number of cosolvent molecules

In [7]:
numsol=numwat*18.0528*(int(conc)/100)/float(pm)
cnumsol=int(math.ceil(numsol))
print(cnumsol)

19


## Calculate Packmol boxes dimensions

In [8]:
intxs=int(math.ceil(float(xsize)))
intys=int(math.ceil(float(ysize)))
intzs=int(math.ceil(float(zheight)))
print("Boxes dimensions:", intxs, intys, intzs)

Boxes dimensions: 39 41 20


## Creates Input for Packmol

In [9]:
output1=conc + "_" + coname + "_box"
packs=output1 + ".inp"
print("""
tolerance 2.5
filetype pdb
output %s.pdb
structure %s.pdb
  number %s
  inside box 0. 0. 0. %s. %s. %s.
end structure
""" %(output1,coname,cnumsol,intxs, intys, intzs), file=open(packs, "w"))

## Runs packmol and creates cosolvent box at desired concentration

In [10]:
os.system(pmol + '<' + packs)

0

## Moves the box to the correct coordinates

In [11]:
cbox=Molecule(output1 + ".pdb")
movedb=cbox.copy()
movedb.moveBy([xmin, ymin , zl2])
movedb.view()
b = VMDBox([xmin, xmax, ymin, ymax, zl2, zl4])
movedb.write(output1 + "_moved.pdb")
cosolvent=(output1 + "_moved.pdb")
print(cosolvent)

10_P2O_box_moved.pdb


## System Setup

In [12]:
# System Setup
prot=Molecule(name + '.pdb')  #needs CYS HG->HG1
prot.set('name','HG1',sel='resname CYS and name HG')
memb=Molecule('membrane_c36.pdb')
memb.filter('resname %s' %(lipid))  #only POPC or embed messes up
prot.set('segid', 'PRO', sel='protein')
prot.set('segid', 'W', sel='water')
prot.set('segid', 'CA', sel='resname CA')
prot.remove('resname ACE')
prot.remove('resname NMA')
csol = Molecule(cosolvent)
csol.set('segid','C1')
prot.append(csol)
mol2=embed(prot,memb)
mol2.view()
mol2.write("embedded_system.pdb")

2021-05-27 16:27:33,065 - htmd.molecule.molecule - INFO - Removed 10611 atoms. 28810 atoms remaining in the molecule.
2021-05-27 16:27:33,364 - htmd.molecule.molecule - INFO - Removed 0 atoms. 4882 atoms remaining in the molecule.
2021-05-27 16:27:33,426 - htmd.molecule.molecule - INFO - Removed 0 atoms. 4882 atoms remaining in the molecule.


## System Solvation

In [13]:
smol2 = solvate(mol2, negz=30, posz=10)
smol2.write('solvated_system.pdb')
smol2.view()

2021-05-27 16:28:06,665 - htmd.builder.solvate - INFO - Using water pdb file at: /usr/local/miniconda3/lib/python3.6/site-packages/htmd/builder/wat.pdb
2021-05-27 16:28:08,138 - htmd.builder.solvate - INFO - Replicating 12 water segments, 2 by 2 by 3
Solvating: 100%|██████████| 12/12 [00:27<00:00,  2.27s/it]
2021-05-27 16:28:43,106 - htmd.builder.solvate - INFO - 28021 water molecules were added to the system.
2021-05-27 16:28:43,413 - htmd.molecule.writers - WARNING - Field "serial" of PDB overflows. Your data will be truncated to 5 characters.
2021-05-27 16:28:49,678 - htmd.molecule.writers - WARNING - Field "serial" of PDB overflows. Your data will be truncated to 5 characters.


## Print Lig e Csolv Par and Top Filenames

In [14]:
csolt="top_" + coname + ".rtf"
csolp="par_" + coname + ".prm"
print(csolt, csolp)

top_P2O.rtf par_P2O.prm


## Create PSF File

In [15]:
topos  = [csolt,'top/top_all36_prot.rtf','top/top_all36_cgenff.rtf','top/top_all36_lipid.rtf','top/top_water_ions.rtf']
params = [csolp,'par/par_all36_prot.prm','par/par_all36_cgenff.prm','par/par_all36_lipid.prm','par/par_water_ions.prm']
plm = charmm.build(smol2, topo=topos, param=params,outdir='build',ionize=True,saltconc=0.154,saltanion='CL',saltcation='NA')

2021-05-27 16:29:16,389 - htmd.builder.charmm - INFO - Writing out segments.
2021-05-27 16:29:37,406 - htmd.molecule.writers - WARNING - Field "serial" of PDB overflows. Your data will be truncated to 5 characters.
2021-05-27 16:29:38,617 - htmd.molecule.writers - WARNING - Field "serial" of PDB overflows. Your data will be truncated to 5 characters.
2021-05-27 16:29:39,678 - htmd.molecule.writers - WARNING - Field "serial" of PDB overflows. Your data will be truncated to 5 characters.
2021-05-27 16:29:40,564 - htmd.molecule.writers - WARNING - Field "serial" of PDB overflows. Your data will be truncated to 5 characters.
2021-05-27 16:29:41,623 - htmd.molecule.writers - WARNING - Field "serial" of PDB overflows. Your data will be truncated to 5 characters.
2021-05-27 16:29:41,717 - htmd.builder.builder - INFO - 2 disulfide bonds were added


Disulfide Bond between: UniqueResidueID<resname: 'CYS', chain: 'A', resid: 42, insertion: '', segid: 'PRO'>
                   and: UniqueResidueID<resname: 'CYS', chain: 'A', resid: 296, insertion: '', segid: 'PRO'>

Disulfide Bond between: UniqueResidueID<resname: 'CYS', chain: 'A', resid: 124, insertion: '', segid: 'PRO'>
                   and: UniqueResidueID<resname: 'CYS', chain: 'A', resid: 202, insertion: '', segid: 'PRO'>



2021-05-27 16:29:42,269 - htmd.builder.charmm - INFO - Starting the build.
2021-05-27 16:29:44,869 - htmd.builder.charmm - WARNING - Failed to set coordinates for 6 atoms.
2021-05-27 16:29:44,870 - htmd.builder.charmm - WARNING - Poorly guessed coordinates for 13 atoms.
2021-05-27 16:29:44,871 - htmd.builder.charmm - WARNING - Please check /media/tianyiding/SeconHD/Archer2/P2Y1_inter/P2O_p2y1_345/test_scripts/build/log.txt for further information.
2021-05-27 16:29:44,872 - htmd.builder.charmm - INFO - Finished building.
2021-05-27 16:29:48,694 - htmd.molecule.readers - WARNING - Non-integer values were read from the PDB "serial" field. Dropping PDB values and assigning new ones.
2021-05-27 16:29:48,861 - htmd.molecule.readers - WARNING - Reading PDB file with more than 99999 atoms. Bond information can be wrong.
2021-05-27 16:29:56,080 - htmd.builder.ionize - INFO - Adding 17 anions + 0 cations for neutralizing and 162 ions for the given salt concentration.
2021-05-27 16:32:30,463 - ht

## View System

In [16]:
plm.view()

2021-05-27 16:34:21,095 - htmd.molecule.writers - WARNING - Field "serial" of PDB overflows. Your data will be truncated to 5 characters.


## Measure Cell Dimensions

In [17]:
boxc=plm.get('coords')
bpmin=np.min(boxc, axis=0)
bpmax=np.max(boxc, axis=0)
bxmin=bpmin[0] 
bxmax=bpmax[0] 
bymin=bpmin[1]
bymax=bpmax[1] 
bzmin=bpmin[2]
bzmax=bpmax[2]
bxsize=bxmax-bxmin 
bysize=bymax-bymin
bzsize=bzmax-bzmin
print(bxsize,bysize,bzsize)

92.906 93.597 142.13


## Measure Cell Origin

In [18]:
bcen=np.average(boxc, axis=0)
print(bcen)
bcenx=bcen[0]
bceny=bcen[1]
bcenz=bcen[2]
print(bcenx,bceny,bcenz)

[-4.23748   -2.5885072  6.054047 ]
-4.23748 -2.5885072 6.054047


## Measure Protein Mass Center

In [19]:
protc=plm.get('coords', sel='protein')
protm=plm.get('masses', sel='protein')
print(protm)
protcm=np.average(protc, axis=0, weights=protm)
print(protcm)
protcenx=protcm[0]
protceny=protcm[1]
protcenz=protcm[2]
print(protcenx,protceny,protcenz)

[12.011  1.008  1.008 ...  1.008  1.008  1.008]
[-0.21692628  0.1851154   1.7042737 ]
-0.21692628 0.1851154 1.7042737


## Create Constraints for Eq01

In [21]:
plm.set('beta', '1', sel='resname %s or water or ions or protein or (resname %s and name P O11 O12 O13 O14 C11 C12 C13 C14 C15 H11A H11B H12A H12B H13A H13B H13C H14A H14B H14C H15A H15B H15C)' %(csolv,lipid))
plm.write('mineq-01.pdb')

2021-02-14 22:26:54,668 - htmd.molecule.writers - WARNING - Field "serial" of PDB overflows. Your data will be truncated to 5 characters.


## Create Constraints for Eq02

In [21]:
plm.set('beta', '0', sel='all')
plm.set('beta', '1', sel='protein')
plm.write('mineq-02.pdb')

2021-02-09 20:42:23,336 - htmd.molecule.writers - WARNING - Field "serial" of PDB overflows. Your data will be truncated to 5 characters.


## Create Constraints for Prod

In [22]:
plm.set('beta', '0', sel='all')
plm.set('beta', '1', sel='protein and name CA and z > -5 and z < 5')
plm.write('prod.pdb')

2021-02-09 20:43:42,316 - htmd.molecule.writers - WARNING - Field "serial" of PDB overflows. Your data will be truncated to 5 characters.


## Create Input File for Eq01

In [37]:
print("""
#############################################################
## JOB DESCRIPTION                                         ##
#############################################################

# Min. and Eq. of Co-Solvent Membrane System
# embedded in POPC membrane, ions and water.
# Melting lipid tails. PME, Constant Volume.

#############################################################
## ADJUSTABLE PARAMETERS                                   ##
#############################################################

structure          structure.psf
coordinates        structure.pdb
outputName         mineq-01

set temperature    310

firsttimestep      0

#############################################################
## SIMULATION PARAMETERS                                   ##
#############################################################
# Input
paraTypeCharmm	    on
parameters          parameter

# NOTE: Do not set the initial velocity temperature if you 
# have also specified a .vel restart file!
temperature         $temperature

# Periodic Boundary Conditions
# NOTE: Do not set the periodic cell basis if you have also 
# specified an .xsc restart file!
if {1} { 
cellBasisVector1     %s    0.   0.
cellBasisVector2     0.    %s   0.
cellBasisVector3     0.    0.   %s
cellOrigin           %s %s %s
}
wrapWater           on
wrapAll             on

# Force-Field Parameters
exclude             scaled1-4
1-4scaling          1.0
cutoff              12.
switching           on
switchdist          10.
pairlistdist        13.5

# Integrator Parameters
timestep            1.0  ;# 1fs/step
rigidBonds          all  ;# needed for 2fs steps
nonbondedFreq       1
fullElectFrequency  2  
stepspercycle       20

#PME (for full-system periodic electrostatics)
if {1} {
PME                yes
PMEGridSpacing     1.0
}

# Constant Temperature Control
langevin            on    ;# do langevin dynamics
langevinDamping     1     ;# damping coefficient (gamma) of 5/ps
langevinTemp        $temperature

restartfreq        1000     ;# 1000steps = every 2ps
dcdfreq            1000
xstFreq            1000
outputEnergies      50
outputPressure      50

# Fixed Atoms Constraint (set PDB beta-column to 1)
if {1} {
fixedAtoms          on
fixedAtomsFile      mineq-01.pdb
fixedAtomsCol       B
fixedAtomsForces    on
}

#############################################################
## EXECUTION SCRIPT                                        ##
#############################################################

# Minimization
if {1} {
minimize            1000
reinitvels          $temperature
}

run 500000;# 0.5 ns
""" %(bxsize, bysize,bzsize,bcenx, bceny, bcenz), file=open("mineq-01.conf", "w"))

In [20]:
csolec=np.unique(plm.get('resid', sel='resname %s and z > 0' %(csolv)))

## Calculate Cylinder Radius

In [21]:
radius=int(math.floor(np.sqrt((xsize*xsize)+(ysize*ysize)))/2)
print(radius)

28


## Calculate Cylinder Smaller Radius

In [22]:
if xsize > ysize:
        smallr=int(math.ceil(xsize)/2)
else:
        smallr=int(math.ceil(ysize)/2)
print(smallr)

20


In [23]:
bigr=(radius+2)
print(bigr)

30


## Create Collective Variable File for Eq02

In [28]:
print("""
Colvarstrajfrequency    50
Colvarsrestartfrequency 50
""", file=open("cosolvent.col", "w"))

for element in csolec:
    zio=plm.get("serial",sel="resname %s and resid %s and z > 0" %(csolv,element))
    print("""
    ### Mol n. %s ###
    colvar {
       name ec_%s
       upperboundary %s
       upperwallconstant 200.0
       lowerboundary 0.0
       lowerwallconstant 200.0
       distancexy {
          ref {
             dummyAtom ( 0.000, 0.000, 0.000 )
          }
          main {
             atomNumbers {%s}
          }
         axis (0.0, 0.0, 1.0)
       }
    }

    colvar {
       name ecz_%s
       upperboundary %s
       lowerboundary 0.0
       upperwallconstant 200.0
       lowerwallconstant 200.0
       distanceZ {
          ref {
             dummyAtom ( 0.000, 0.000, %s )
          }
          main {
            atomNumbers {%s}
          }
       }
    }
""" %(element,element,radius,str(zio).strip('[]'),element,zheight,zl2,str(zio).strip('[]')), file=open("cosolvent.col", "a"))

## Create Input File for Eq02

In [29]:
print("""
#############################################################
## JOB DESCRIPTION                                         ##
#############################################################

# Min. and Eq. of Co-Solvent Membrane System
# embedded in POPC membrane, ions and water.
# Protein constrained. PME, Constant Pressure.

#############################################################
## ADJUSTABLE PARAMETERS                                   ##
#############################################################

structure          structure.psf
coordinates        structure.pdb

outputName         mineq-02

set temperature    310

# Continuing a job from the restart files
if {1} {
set inputname      mineq-01
binCoordinates     ./$inputname.restart.coor
binVelocities      ./$inputname.restart.vel  
extendedSystem	   ./$inputname.restart.xsc
} 

firsttimestep      501000 ;# last step of previous run

#############################################################
## SIMULATION PARAMETERS                                   ##
#############################################################

# Input
paraTypeCharmm	    on
parameters          parameters

wrapWater           on
wrapAll             on

# Force-Field Parameters
exclude             scaled1-4
1-4scaling          1.0
cutoff              12.
switching           on
switchdist          10.
pairlistdist        13.5

# Integrator Parameters
timestep            2.0  ;# 2fs/step
rigidBonds          all  ;# needed for 2fs steps
nonbondedFreq       1
fullElectFrequency  2  
stepspercycle       20

#PME (for full-system periodic electrostatics)
if {1} {
PME                yes
PMEGridSpacing     1.0
}

# Constant Temperature Control
langevin            on    ;# do langevin dynamics
langevinDamping     1     ;# damping coefficient (gamma) of 5/ps
langevinTemp        $temperature

# Constant Pressure Control (variable volume)
if {1} {
useGroupPressure      yes ;# needed for 2fs steps
useFlexibleCell       yes  ;# no for water box, yes for membrane
useConstantArea       no  ;# no for water box, yes for membrane

langevinPiston        on
langevinPistonTarget  1.01325 ;#  in bar -> 1 atm
langevinPistonPeriod  200.
langevinPistonDecay   50.
langevinPistonTemp    $temperature
}

restartfreq        1000     ;# 1000steps = every 2ps
dcdfreq            1000
xstFreq            1000
outputEnergies      50
outputPressure      50

#############################################################
## EXTRA PARAMETERS                                        ##
#############################################################

# Put here any custom parameters that are specific to 
# this job (e.g., SMD, TclForces, etc...)

constraints on
consexp 2
consref ./structure.pdb
conskfile mineq-02.pdb       
conskcol B
margin 3

tclforces on
set waterCheckFreq              100
set lipidCheckFreq              100
set allatompdb                  ./structure.pdb
tclForcesScript                 keep_water_out.tcl

colvars on
colvarsConfig cosolvent.col

#############################################################
## EXECUTION SCRIPT                                        ##
#############################################################

# Minimization
if {1} {
minimize            500
reinitvels          $temperature
}

run 1000000 ;# 2.0 ns

""", file=open("mineq-02.conf", "w"))

## Check if there are gaps in the sequence and adjust the CA accordingly

In [24]:
# Copy cosolvent.col and append a line for the protein
dups = prot.copy()                       # Make a working copy
dups.filter("name CA and protein")      # Keep only C-alphas
rid = dups.get('resid')                 # Get list of residue ids
nrid, count= np.unique(rid,return_counts=True) # Get how many times each residue id appeared
nrid[count>1]                           # Show duplicates
deltas = np.diff(rid)
print(deltas)
new_rid = rid[:np.size(rid)-1] # no delta for last residue
new_rid[deltas!=1]
rn = dups.get('resname')
for i in range(np.size(rid)-1):
    if(deltas[i]>1):
        print(rid[i],rn[i],' followed by ',rid[i+1],rn[i+1])
        print('%s-%s,'%(rid[i],rid[i+1]))
        print('%s-%s,%s-%s'%(rid[0],rid[i],rid[i+1],rid[-1]))
        ca=str('%s-%s,%s-%s'%(rid[0],rid[i],rid[i+1],rid[-1]))
        print(ca)

2021-05-27 16:38:06,430 - htmd.molecule.molecule - INFO - Removed 4813 atoms. 297 atoms remaining in the molecule.


[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [25]:
ca=('%s-%s'%(nrid[0],nrid[-1]))
print(ca)

38-334


ca=('19-217,376-457')
print(ca)

## Create Collective Variable File for Eq03

In [32]:
shutil.copy2('cosolvent.col', 'cosolvent2.col')
print("""
colvar {
  name Origin
  distance {
     group1 {
      psfSegID PRO
      atomNameResidueRange CA %s
      centerReference on
      refPositions (0,0,0)
     }
     group2 {
     dummyAtom (%s,%s,%s)
     }
  }
}
""" %(ca,protcenx,protceny,protcenz), file=open("cosolvent2.col", "a"))

## Create Input File for Eq03

In [33]:
print("""
#############################################################
## JOB DESCRIPTION                                         ##
#############################################################

# Min. and Eq. of Co-Solvent Membrane System
# embedded in POPC membrane, ions and water.
# Protein released. PME, Constant Pressure.

#############################################################
## ADJUSTABLE PARAMETERS                                   ##
#############################################################

structure          structure.psf
coordinates        structure.pdb
outputName         eq-03

set temperature    310

# Continuing a job from the restart files
if {1} {
set inputname      mineq-02
binCoordinates     ./$inputname.restart.coor
binVelocities      ./$inputname.restart.vel  
extendedSystem	   ./$inputname.restart.xsc
} 

firsttimestep      1501500 ;# last step of previous run

#############################################################
## SIMULATION PARAMETERS                                   ##
#############################################################

# Input
paraTypeCharmm	    on
parameters          parameters

wrapWater           on
wrapAll             on

# Force-Field Parameters
exclude             scaled1-4
1-4scaling          1.0
cutoff              12.
switching           on
switchdist          10.
pairlistdist        13.5

# Integrator Parameters
timestep            2.0  ;# 2fs/step
rigidBonds          all  ;# needed for 2fs steps
nonbondedFreq       1
fullElectFrequency  2  
stepspercycle       20

#PME (for full-system periodic electrostatics)
if {1} {
PME                yes
PMEGridSpacing     1.0
}

# Constant Temperature Control
langevin            on    ;# do langevin dynamics
langevinDamping     1     ;# damping coefficient (gamma) of 5/ps
langevinTemp        $temperature

# Constant Pressure Control (variable volume)
if {1} {
useGroupPressure      yes ;# needed for 2fs steps
useFlexibleCell       yes  ;# no for water box, yes for membrane
useConstantArea       no  ;# no for water box, yes for membrane

langevinPiston        on
langevinPistonTarget  1.01325 ;#  in bar -> 1 atm
langevinPistonPeriod  200.
langevinPistonDecay   50.
langevinPistonTemp    $temperature
}

restartfreq        1000     ;# 1000steps = every 2ps
dcdfreq            1000
xstFreq            1000
outputEnergies      50
outputPressure      50

#############################################################
## EXTRA PARAMETERS                                        ##
#############################################################

# Put here any custom parameters that are specific to 
# this job (e.g., SMD, TclForces, etc...)

colvars on
colvarsConfig cosolvent2.col

#############################################################
## EXECUTION SCRIPT                                        ##
#############################################################

run 5000000;# 10.0 ns
""", file=open("eq-03.conf", "w"))

In [26]:
bigr2=(bigr+5)
print(bigr2)

35


In [27]:
zheight2="10.0000"
print(zheight, zheight2)

20.0000 10.0000


## Create Collective Variable File for Production

In [28]:
print("""
Colvarstrajfrequency    50
Colvarsrestartfrequency 50
""", file=open("cosolvent_helices_345_interface.col", "w"))

for element in csolec:
    zio=plm.get("serial",sel="resname %s and resid %s and z > 0" %(csolv,element))
    print("""
    ### Mol n. %s ###
    colvar {
       name ec_%s
       upperboundary %s
       upperwall %s
       upperwallconstant 200.0
       lowerboundary 10.0
       lowerwall 10.0
       lowerwallconstant 200.0
       distancexy {
          ref {
             dummyAtom ( 0.000, 0.000, 0.000 )
          }
          main {
             atomNumbers {%s}
          }
         axis (0.0, 0.0, 1.0)
       }
    }

    colvar {
       name ecz_%s
       upperboundary %s
       lowerboundary 0.0
       upperwall %s
       upperwallconstant 200.0
       distanceZ {
          ref {
             dummyAtom ( 0.000, 0.000, %s )
          }
          main {
            atomNumbers {%s}
          }
       }
    } 
    
    """ %(element,element,bigr2,bigr2,str(zio).strip('[]'),element,zheight2,zheight2,zl2,str(zio).strip('[]')), file=open("cosolvent345.col", "a"))

for element in csolec:
    zio=plm.get("serial",sel="resname %s and resid %s and z > 0" %(csolv,element))
    print("""
    ### Collective Variables for Half Cylinder ###
    ### Mol n. %s ###
    colvar {
       name ecx_%s
       upperboundary %s
       upperwall %s
       upperwallconstant 200.0
       lowerboundary 10.0
       lowerwall 10.0
       lowerwallconstant 200.0
       distanceZ {
          ref {
             dummyAtom ( 0.000, 0.000, -%s )
          }
          main {
             atomNumbers {%s}
          }
         axis (1.0, 0.0, 0.0)
       }
    }
    ### Collective Variables for Sector ###
    colvar {
       name ecy_%s
       upperboundary %s
       upperwall %s
       lowerboundary 0.0
       lowerwall 0.0
       upperwallconstant 200.0
       lowerwallconstant 200.0
       distanceZ {
          ref {
             dummyAtom ( 0.000, 0.000, 0.000)
          }
          main {
            atomNumbers {%s}
          }
          axis (0.0, -1.0, 0.0)
       }
    } 
    
    """ %(element,element,bigr2,bigr2,bigr2,str(zio).strip('[]'),element,bigr2,bigr2,str(zio).strip('[]')), file=open("cosolvent345.col", "a"))
print("""
    colvar {
      name Origin
      distance {
         group1 {
          psfSegID PRO
          atomNameResidueRange CA %s
          centerReference on
          refPositions (0,0,0)
         }
         group2 {
         dummyAtom (%s,%s,%s)
         }
      }
    }
    """%(ca,protcenx,protceny,protcenz), file=open("cosolvent_helices_345_interface.col", "a"))

In [29]:
print("""
Colvarstrajfrequency    50
Colvarsrestartfrequency 50
""", file=open("cosolvent_helices_234_interface.col", "w"))

for element in csolec:
     zio=plm.get("serial",sel="resname %s and resid %s and z > 0" %(csolv,element))
    print("""
    ### Mol n. %s ###
    colvar {
       name ec_%s
       upperboundary %s
       upperwall %s
       upperwallconstant 200.0
       lowerboundary 10.0
       lowerwall 10.0
       lowerwallconstant 200.0
       distancexy {
          ref {
             dummyAtom ( 0.000, 0.000, 0.000 )
          }
          main {
             atomNumbers {%s}
          }
         axis (0.0, 0.0, 1.0)
       }
    }

    colvar {
       name ecz_%s
       upperboundary %s
       lowerboundary 0.0
       upperwall %s
       upperwallconstant 200.0
       distanceZ {
          ref {
             dummyAtom ( 0.000, 0.000, %s )
          }
          main {
            atomNumbers {%s}
          }
       }
    } 
    
    """ %(element,element,bigr2,bigr2,str(zio).strip('[]'),element,zheight2,zheight2,zl2,str(zio).strip('[]')), file=open("cosolvent234.col", "a"))

for element in csolec:
    zio=plm.get("serial",sel="resname %s and resid %s and z > 0" %(csolv,element))
    print("""
    ### Collective Variables for Half Cylinder ###
    ### Mol n. %s ###
    colvar {
       name ecx_%s
       upperboundary %s
       upperwall %s
       upperwallconstant 200.0
       lowerboundary 10.0
       lowerwall 10.0
       lowerwallconstant 200.0
       distanceZ {
          ref {
             dummyAtom ( 0.000, 0.000, -%s )
          }
          main {
             atomNumbers {%s}
          }
         axis (-1.0, 0.0, 0.0)
       }
    }
    ### Collective Variables for Sector ###
    colvar {
       name ecy_%s
       upperboundary %s
       upperwall %s
       lowerboundary 0.0
       lowerwall 0.0
       upperwallconstant 200.0
       lowerwallconstant 200.0
       distanceZ {
          ref {
             dummyAtom ( 0.000, 0.000, 0.000)
          }
          main {
            atomNumbers {%s}
          }
          axis (0.0, -1.0, 0.0)
       }
    } 
    
    """ %(element,element,bigr2,bigr2,bigr2,str(zio).strip('[]'),element,bigr2,bigr2,str(zio).strip('[]')), file=open("cosolvent234.col", "a"))
print("""
    colvar {
      name Origin
      distance {
         group1 {
          psfSegID PRO
          atomNameResidueRange CA %s
          centerReference on
          refPositions (0,0,0)
         }
         group2 {
         dummyAtom (%s,%s,%s)
         }
      }
    }
    """%(ca,protcenx,protceny,protcenz), file=open("cosolvent_helices_234_interface.col", "a"))

In [30]:
print("""
Colvarstrajfrequency    50
Colvarsrestartfrequency 50
""", file=open("cosolvent_helices_67_interface.col", "w"))

for element in csolec:
    zio=plm.get("serial",sel="resname %s and resid %s and z > 0" %(csolv,element))
    print("""
    ### Mol n. %s ###
    colvar {
       name ec_%s
       upperboundary %s
       upperwall %s
       upperwallconstant 200.0
       lowerboundary 10.0
       lowerwall 10.0
       lowerwallconstant 200.0
       distancexy {
          ref {
             dummyAtom ( 0.000, 0.000, 0.000 )
          }
          main {
             atomNumbers {%s}
          }
         axis (0.0, 0.0, 1.0)
       }
    }

    colvar {
       name ecz_%s
       upperboundary %s
       lowerboundary 0.0
       upperwall %s
       upperwallconstant 200.0
       distanceZ {
          ref {
             dummyAtom ( 0.000, 0.000, %s )
          }
          main {
            atomNumbers {%s}
          }
       }
    } 
    
    """ %(element,element,bigr2,bigr2,str(zio).strip('[]'),element,zheight2,zheight2,zl2,str(zio).strip('[]')), file=open("cosolvent67.col", "a"))

for element in csolec:
    zio=plm.get("serial",sel="resname %s and resid %s and z > 0" %(csolv,element))
    print("""
    ### Collective Variables for Half Cylinder ###
    ### Mol n. %s ###
    colvar {
       name ecx_%s
       upperboundary %s
       upperwall %s
       upperwallconstant 200.0
       lowerboundary -5.0
       lowerwall -5.0
       lowerwallconstant 200.0
       distanceZ {
          ref {
             dummyAtom ( 0.000, 0.000, -%s )
          }
          main {
             atomNumbers {%s}
          }
         axis (1.0, 0.0, 0.0)
       }
    }
    ### Collective Variables for Sector ###
    colvar {
       name ecy_%s
       upperboundary %s
       upperwall %s
       lowerboundary 0.0
       lowerwall 0.0
       upperwallconstant 200.0
       lowerwallconstant 200.0
       distanceZ {
          ref {
             dummyAtom ( 0.000, 0.000, 0.000)
          }
          main {
            atomNumbers {%s}
          }
          axis (0.0, 1.0, 0.0)
       }
    } 
    
    """ %(element,element,bigr2,bigr2,bigr2,str(zio).strip('[]'),element,bigr2,bigr2,str(zio).strip('[]')), file=open("cosolvent67.col", "a"))
print("""
    colvar {
      name Origin
      distance {
         group1 {
          psfSegID PRO
          atomNameResidueRange CA %s
          centerReference on
          refPositions (0,0,0)
         }
         group2 {
         dummyAtom (%s,%s,%s)
         }
      }
    }
    """%(ca,protcenx,protceny,protcenz), file=open("cosolvent_helices_67_interface.col", "a"))

## Create Input File for Production

In [31]:
print("""
#############################################################
## JOB DESCRIPTION                                         ##
#############################################################

# Min. and Eq. of Co-Solvent Membrane System
# embedded in POPC membrane, ions and water.
# Protein released. PME, Constant Pressure and Area.

#############################################################
## ADJUSTABLE PARAMETERS                                   ##
#############################################################

structure          structure.psf
coordinates        structure.pdb
outputName         run8          

set temperature    310

# Continuing a job from the restart files
if {1} {
set inputname      eq-03
binCoordinates     ./$inputname.restart.coor
binVelocities      ./$inputname.restart.vel  
extendedSystem	   ./$inputname.restart.xsc
} 

firsttimestep      6501500 ;# last step of previous run

#############################################################
## SIMULATION PARAMETERS                                   ##
#############################################################

# Input
paraTypeCharmm	    on
parameters          parameters_prod
parameters          par_P2O.prm

wrapWater           on
wrapAll             on

# Force-Field Parameters
exclude             scaled1-4
1-4scaling          1.0
cutoff              12.
switching           on
switchdist          10.
pairlistdist        13.5

# Integrator Parameters
timestep            2.0  ;# 2fs/step
rigidBonds          all  ;# needed for 2fs steps
nonbondedFreq       1
fullElectFrequency  2  
stepspercycle       20

#PME (for full-system periodic electrostatics)
if {1} {
PME                yes
PMEGridSpacing     1.0
}

# Constant Temperature Control
langevin            on    ;# do langevin dynamics
langevinDamping     1     ;# damping coefficient (gamma) of 5/ps
langevinTemp        $temperature

# Constant Pressure Control (variable volume)
if {1} {
useGroupPressure      yes ;# needed for 2fs steps
useFlexibleCell       yes  ;# no for water box, yes for membrane
useConstantArea       yes ;# no for water box, yes for membrane

langevinPiston        on
langevinPistonTarget  1.01325 ;#  in bar -> 1 atm
langevinPistonPeriod  200.
langevinPistonDecay   50.
langevinPistonTemp    $temperature
}

restartfreq        1000     ;# 1000steps = every 2ps
dcdfreq            1000
xstFreq            1000
outputEnergies      50
outputPressure      50

#############################################################
## EXTRA PARAMETERS                                        ##
#############################################################

# Put here any custom parameters that are specific to 
# this job (e.g., SMD, TclForces, etc...)

constraints on
consexp 2 
consref ./structure.pdb
conskfile prod.pdb
conskcol B
margin 3  

colvars on
colvarsConfig cosolvent3.col

#############################################################
## EXECUTION SCRIPT                                        ##
#############################################################
if {1} { 
  minimize            1800
  reinitvels          $temperature
  } 

  run 20000000 ;# 40.0 ns
""",file=open("run.conf", "w"))

## Print System Specs in a file

In [32]:
print("""
Protein Dimensions: %s %s
Cosolvent Boxes Dimensions: %s %s %s
Number of Cosolvent Molecules: %s
Cell Dimensions: %s %s %s
Cell Origin: %s %s %s
Protein Center of Mass: %s %s %s
Cylinder Radius: %s
Cylinder Z Boundary: %s
Cylinder Bigger Radius: %s
New Cylinder Bigger Radius: %s
""" %(xsize,ysize,intxs,intys,intzs,cnumsol,bxsize,bysize,bzsize,bcenx,bceny,bcenz,protcenx,protceny,protcenz,radius,zl2,bigr,bigr2),file=open("system-specs.txt", "w"))